In [ ]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
#Load data
df=pd.read_csv('.\data.\processed_EPS1.csv',index_col=0)
df

In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor,TabNetClassifier

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error,accuracy_score

import pandas as pd
import numpy as np
np.random.seed(0)


import os
import wget
from pathlib import Path

In [ ]:
df2 = df.drop(['D1', 'D0', 'Validity', 'ipk', 'irms', 'Vo', 'Ptotal'], axis=1)
df2 = df2.reset_index(drop=True)

In [ ]:
df2

In [ ]:
target = 'nZVS'
#Divide the dataset into training set, validation set and test set
if "Set" not in df2.columns:
    df2["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(df2.shape[0],))
train_indices = df2[df2.Set=="train"].index
valid_indices = df2[df2.Set=="valid"].index
test_indices = df2[df2.Set=="test"].index

In [ ]:
tabnet_params = { "optimizer_fn":torch.optim.Adam,
                 "optimizer_params":dict(lr=0.02),
                 "scheduler_params": {"mode": 'max', "factor": 0.5, "patience": 30, "verbose": True},
                 "scheduler_fn":torch.optim.lr_scheduler.ReduceLROnPlateau,
                 "n_d":32,
                 "n_a":32,
                 "n_steps":5,
                 "n_independent": 5,
                # "mask_type":'entmax', # "sparsemax"
                # "grouped_features" : grouped_features
                }


In [ ]:
clf = TabNetClassifier(n_d=16,n_a=16)

In [ ]:
unused_feat1 = ['Set']
features1 = [ col for col in df2.columns if col not in unused_feat1+[target]] 

In [ ]:
X_train = df2[features1].values[train_indices]
y_train = df2[target].values[train_indices]

X_valid = df2[features1].values[valid_indices]
y_valid = df2[target].values[valid_indices]

X_test = df2[features1].values[test_indices]
y_test = df2[target].values[test_indices]

In [ ]:
y_train

In [ ]:
nZVS_train=(y_train-4)/2
nZVS_train

In [ ]:
nZVS_valid=(y_valid-4)/2
nZVS_test=(y_test-4)/2

In [ ]:
nZVS_train1=torch.tensor(nZVS_train)
nZVS_valid1=torch.tensor(nZVS_valid)
nZVS_test1=torch.tensor(nZVS_test)

In [ ]:
max_epochs = 600 if not os.getenv("CI", False) else 2

In [ ]:
clf.fit(
    X_train=X_train, y_train=nZVS_train,
    eval_set=[(X_train, nZVS_train), (X_valid, nZVS_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    max_epochs=max_epochs,
    patience=200,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    #augmentations=aug
) 

In [ ]:
preds = clf.predict(X_test)

y_true = nZVS_test

test_score = accuracy_score(y_pred=preds, y_true=y_true)
test_score

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define the Random Forest Model
rf_model = RandomForestClassifier()

# Define the Hyperparameter Grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
}

# Hyperparameter tuning with grid searc
grid = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy')

grid.fit(X_train, nZVS_train)

# Get the best model
best_model = grid.best_estimator_

# Prediction on the test set
nZVS_test_pred = best_model.predict(X_test)

# Calculate the MSE and MAE on the test set
accuracy = accuracy_score(nZVS_test, nZVS_test_pred)
report = classification_report(nZVS_test, nZVS_test_pred)

print(f"Best Parameters: {grid.best_params_}")
print(f"Accuracy on Test Set: {accuracy:.4f}")
print(report)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define the XGBoost Model
xgb_model = xgb.XGBClassifier()


param_grid = {
    'n_estimators': [50,100,200],  
    'max_depth': [4, 5,6],  

}


grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, nZVS_train)


print(f"Best Parameters: {grid_search.best_params_}")

best_model = grid.best_estimator_


nZVS_test_pred = best_model.predict(X_test)


accuracy = accuracy_score(nZVS_test, nZVS_test_pred)
print(f"Accuracy on Test Set: {accuracy:.4f}")


In [ ]:
scaler_inputs=MinMaxScaler()
scaler_inputs.fit(inputs_train)
inputs_train=scaler_inputs.transform(inputs_train)
inputs_train=torch.tensor(inputs_train)
inputs_train

In [ ]:
scaler_inputs=MinMaxScaler()
scaler_inputs.fit(inputs_valid)
inputs_valid=scaler_inputs.transform(inputs_valid)
inputs_valid=torch.tensor(inputs_valid)
inputs_valid

In [ ]:
scaler_inputs=MinMaxScaler()
scaler_inputs.fit(inputs_test)
inputs_test=scaler_inputs.transform(inputs_test)
inputs_test=torch.tensor(inputs_test)
inputs_test

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


# Define the SVR Model
model = SVC()


param_grid = {
    'C': [1,10, 100],
    'kernel': ['linear',  'rbf'],
}


grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(inputs_train, nZVS_train)


best_model1 = grid_search.best_estimator_
best_params = grid_search.best_params_


print(f"Best Parameters: {best_params}")



nZVS_test_pred = best_model1.predict(inputs_test)

accuracy = accuracy_score(nZVS_test, nZVS_test_pred)
report = classification_report(nZVS_test, nZVS_test_pred)


print(f"Accuracy on Test Set: {accuracy:.4f}")
print(report)

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import copy

In [ ]:
nZVS_train1=torch.tensor(nZVS_train)
nZVS_valid1=torch.tensor(nZVS_valid)
nZVS_test1=torch.tensor(nZVS_test)

In [ ]:
train_dataset = TensorDataset(inputs_train,nZVS_train1)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
valid_dataset = TensorDataset(inputs_valid,nZVS_valid1)
valid_loader = DataLoader(valid_dataset, batch_size=1024, shuffle=True)
test_dataset = TensorDataset(inputs_test,nZVS_test1)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=True)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math,copy,time
from torch.autograd import Variable

In [ ]:
class Single_Residual1(nn.Module):
    def __init__(self,input_size,hidden_size,dropout,final=False):
        super(Single_Residual1,self).__init__()
        self.linear_layer=nn.Linear(input_size,hidden_size)
        self.final=final
        if not self.final:
            self.batch_norm=nn.BatchNorm1d(hidden_size)
            self.dropout=nn.Dropout(dropout)
        self.residual_maker=nn.Parameter(torch.zeros(input_size,hidden_size))
        self.residual_maker.requires_grad_(False)
        if hidden_size<=input_size:
            self.residual_maker[torch.randperm(input_size)[:hidden_size],torch.arange(0,hidden_size)]=1
        else:
            self.residual_maker[torch.arange(0,input_size),torch.randperm(hidden_size)[:input_size]] = 1
            
    def forward(self,x):
        if self.final:
            h1=self.linear_layer(x)
        else:
            h1=F.relu(self.linear_layer(x))
        return h1

class Residual_DNN1(nn.Module):
    def __init__(self,input_size,hidden_sizes,dropout,num_classes):
        super(Residual_DNN1,self).__init__()
        assert isinstance(hidden_sizes,list),'the "hidden_sizes" should be list '
        hidden_num=len(hidden_sizes)
        self.residuals=[f"self.residual{i}"for i in range(hidden_num)]
        for i in range(hidden_num):
            if i==hidden_num-1:
                exec(self.residuals[i]+"=Single_Residual1(input_size,hidden_sizes[i],dropout,True)")
            else:
                exec(self.residuals[i]+"=Single_Residual1(input_size,hidden_sizes[i],dropout)")
            input_size=hidden_sizes[i]
        self.output_layer = nn.Linear(input_size, num_classes)
    def forward(self,x):
        for residual in self.residuals:
            x=eval(residual)(x)
        x = self.output_layer(x)
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Initialize model
input_size = 3
hidden_sizes = [64, 128, 256, 3]
dropout = 0.1
num_classes = 3
model = Residual_DNN1(input_size, hidden_sizes, dropout, num_classes)
model.train()
epochs = 200

# Define the loss function and optimiser
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=15, verbose=True)

# Model training
best_accuracy = 0.0
best_model_state = None
early_stopping_patience = 80
epochs_no_improve = 0

for epoch in range(epochs):
    for inputs, targets in train_loader:
        inputs = inputs.float()
        targets = targets.squeeze().long()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Evaluate model performance
    model.eval()
    all_predictions = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in valid_loader:  
            inputs = inputs.float()
            targets = targets.squeeze().long()
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.numpy())
            all_targets.extend(targets.numpy())

    # Calculate accuracy
    accuracy = accuracy_score(all_targets, all_predictions)
    print(f'Epoch {epoch}: Validation Accuracy = {accuracy}')
    scheduler.step(accuracy)

    # Check and save the best model
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model_state = model.state_dict()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    
    if epochs_no_improve == early_stopping_patience:
        print(f'Early stopping triggered after {epoch + 1} epochs.')
        break

# load the best model
model.load_state_dict(best_model_state)
model.eval()

#Calculate the MSE and MAE on the test set
all_test_predictions = []
all_test_targets = []
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.float()
        targets = targets.squeeze().long()
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_test_predictions.extend(predicted.numpy())
        all_test_targets.extend(targets.numpy())

test_accuracy = accuracy_score(all_test_targets, all_test_predictions)
print(f'Test Accuracy with best model: {test_accuracy}')

